In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from numpy.linalg import norm
import keras
from keras.layers import Activation, Dense
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import roc_auc_score, accuracy_score
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.utils import to_categorical
from google.colab import drive
drive.mount('/content/drive')

Instructions for updating:
non-resource variables are not supported in the long term


Mounted at /content/drive


In [ ]:
import numpy as np
import scipy.io

m = scipy.io.loadmat('/content/drive/MyDrive/datasets/BCICIV_1_mat/BCICIV_calib_ds1d.mat', struct_as_record=True)

In [ ]:
iris=pd.read_csv('/content/drive/MyDrive/Iris.csv')
xvals = iris[['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']].astype(np.float32)
yvals = iris['Species']
yvals=to_categorical(np.asarray(yvals.factorize()[0]))

In [56]:
def func_modified(regularizer_rate_0,regularizer_rate_1,num_layers_0, epochs, batch_size, num_classes, sensor_sizes, dep):

  from numpy.linalg import norm
  import keras
  from keras.layers import Activation, Dense
  import numpy as np
  import pandas as pd
  import seaborn as sns
  import matplotlib.pyplot as plt
  import tensorflow as tf
  from sklearn.metrics import roc_auc_score, accuracy_score, precision_score
  from tensorflow.keras.layers import Dense, Activation
  from tensorflow.keras.utils import to_categorical
  import tensorflow.compat.v1 as tf
  tf.disable_v2_behavior()
  from sklearn.model_selection import train_test_split
  

  xvals_train, xvals_test,yvals_train, yvals_test = train_test_split(xvals,yvals,
                                   random_state=104, 
                                   test_size=0.2, 
                                   shuffle=True)
  

  starter_learning_rate = 0.001
  num_features=sum(sensor_sizes)
  nrow=len(yvals_train)
  num_output=num_classes

  input_X = tf.placeholder('float32',shape =(None,num_features),name="input_X")
  input_y = tf.placeholder('float32',shape = (None,num_classes),name='input_Y')

  s=tf.compat.v1.InteractiveSession()
  ## Weights initialized by random normal function with std_dev = 1/sqrt(number of input features)
  weights_0 = tf.Variable(tf.random.normal([num_features,num_layers_0], stddev=(1/tf.sqrt(float(num_features)))))
  bias_0 = tf.Variable(tf.random.normal([num_layers_0]))
  weights_1 = tf.Variable(tf.random.normal([num_layers_0,num_output], stddev=(1/tf.sqrt(float(num_layers_0)))))
  bias_1 = tf.Variable(tf.random.normal([num_output]))

  ## Initializing weigths and biases
  hidden_output_0 = tf.nn.relu(tf.matmul(input_X,weights_0)+bias_0)
  predicted_y = tf.sigmoid(tf.matmul(hidden_output_0,weights_1) + bias_1)

  ##calculate penalty term
  series = pd.Series(sensor_sizes)
  cumsum = series.cumsum()
  penalty=(tf.reduce_sum(tf.square(weights_0[0:sensor_sizes[0]])))**0.5
  for i in range(len(sensor_sizes)-1):
    penalty=penalty+(tf.reduce_sum(tf.square(weights_0[cumsum[i]:cumsum[i+1]])))**0.5

  cumsum =[0]+ list(series.cumsum())
  red=0
  r_mat=np.array(xvals_train.corr())
  rsq_mat=[[elem*elem for elem in inner] for inner in r_mat]
  rsq_mat=pd.DataFrame(rsq_mat)
  for i in range(len(sensor_sizes)):
    for j in range(len(sensor_sizes)):
      if j!=i:
        red=red+dep(rsq_mat,sensor_sizes,i+1,j+1)*((tf.reduce_sum(tf.square(weights_0[cumsum[j]:cumsum[j+1]])))*(tf.reduce_sum(tf.square(weights_0[cumsum[i]:cumsum[i+1]])))**0.5)
    
  red=red/(len(sensor_sizes)*(len(sensor_sizes)-1))

  loss = tf.reduce_mean(tf.square(predicted_y-tf.convert_to_tensor(yvals_train, dtype=tf.float32))) + regularizer_rate_0*red + regularizer_rate_1*penalty 


  ## Variable learning rate
  learning_rate = tf.train.exponential_decay(starter_learning_rate, 0, 5, 0.85, staircase=True)
  ## Adam optimzer for finding the right weight
  optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss,var_list=[weights_0,weights_1,
                                                                         bias_0,bias_1])    
  ## Metrics definition
  correct_prediction = tf.equal(tf.argmax(yvals_train,1), tf.argmax(predicted_y,1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  training_accuracy = []
  training_loss = []
  testing_accuracy = []
  #s.run(tf.initialize_all_variables)
  s.run(tf.compat.v1.global_variables_initializer())
  for epoch in range(epochs):    
    arr = np.arange(nrow)
    np.random.shuffle(arr)
    for index in range(0,nrow,batch_size):
        s.run(optimizer, {input_X: xvals_train,
                          input_y: yvals_train})
        
    training_accuracy.append(s.run(accuracy, feed_dict= {input_X:xvals_train, 
                                                         input_y: yvals_train}))
    training_loss.append(s.run(loss, {input_X: xvals_train, 
                                      input_y: yvals_train}))
    print("Epoch:{0}, Train loss: {1:.2f} Train acc: {2:.3f}".format(epoch,
                                                                    training_loss[epoch],
                                                                    training_accuracy[epoch]))
   
  ## Evaluation of test data
  for i in range(len(xvals_test)):
    print('Actual:', yvals_test[i], 'Predicted:', np.rint(s.run(predicted_y, feed_dict={input_X: [xvals_test.iloc[i]]})))

  y_pred = np.rint(s.run(predicted_y, feed_dict={input_X: xvals_test}))
  print(len(y_pred))
  print(len(yvals_test))
  score = accuracy_score(yvals_test, y_pred)
 
  print("\nTest Accuracy: {0:f}\n".format(score))

  w0=weights_0.eval()
  w=[]
  #w.append(norm(w0[0:sensor_sizes[0]],2))
  for i in range(len(sensor_sizes)):
    w.append(norm(w0[cumsum[i]:cumsum[i+1]],2))
  print(w)
  s.close()
  return([weights_0,weights_1, bias_0,bias_1])

In [ ]:
func_modified(0,0,10,500,10,3,[2,2],dep_cor)

/usr/local/lib/python3.8/dist-packages/tensorflow/python/client/session.py:1768: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Epoch:0, Train loss: 0.45 Train acc: 0.317
Epoch:1, Train loss: 0.42 Train acc: 0.317
Epoch:2, Train loss: 0.40 Train acc: 0.317
Epoch:3, Train loss: 0.38 Train acc: 0.317
Epoch:4, Train loss: 0.37 Train acc: 0.317
Epoch:5, Train loss: 0.37 Train acc: 0.317
Epoch:6, Train loss: 0.36 Train acc: 0.317
Epoch:7, Train loss: 0.36 Train acc: 0.317
Epoch:8, Train loss: 0.35 Train acc: 0.317
Epoch:9, Train loss: 0.35 Train acc: 0.317
Epoch:10, Train loss: 0.34 Train acc: 0.317
Epoch:11, Train loss: 0.34 Train acc: 0.317
Epoch:12, Train loss: 0.32 Train acc: 0.317
Epoch:13, Train loss: 0.29 Train acc: 0.317
Epoch:14, Train loss: 0.24 Train acc: 0.608
Epoch:15, Train loss: 0.18 Train acc: 0.683
Epoch:16, Train loss: 0.15 Train acc: 0.683
Epoch:17, Train loss: 0.14 Train acc: 0.683
Epoch:18, Train loss: 0.14 Train acc: 0.683
Epoch:19, Train loss: 0.13 Train acc: 0.683
Epoch:20, Train loss: 0.13 Train acc: 0.692
Epoch:21, Train loss: 0.12 Train acc: 0.700
Epoch:22, Train loss: 0.12 Train acc: 0.77

[<tf.Variable 'Variable_32:0' shape=(4, 10) dtype=float32_ref>,
 <tf.Variable 'Variable_34:0' shape=(10, 3) dtype=float32_ref>,
 <tf.Variable 'Variable_33:0' shape=(10,) dtype=float32_ref>,
 <tf.Variable 'Variable_35:0' shape=(3,) dtype=float32_ref>]

In [ ]:
def func_modified2(regularizer_rate_0,regularizer_rate_1,num_layers_0, epochs, batch_size, num_classes, sensor_sizes, dep, xvals, yvals, reduction):

  from numpy.linalg import norm
  import keras
  from keras.layers import Activation, Dense
  import numpy as np
  import pandas as pd
  import tensorflow as tf
  from sklearn.metrics import accuracy_score
  from tensorflow.keras.layers import Dense, Activation
  import tensorflow.compat.v1 as tf
  tf.disable_v2_behavior()
  from sklearn.model_selection import train_test_split

  xvals_train, xvals_test,yvals_train, yvals_test = train_test_split(xvals,yvals,random_state=None, test_size=0.2,  shuffle=True)
                                                                     
  starter_learning_rate = 0.001
  num_features=sum(sensor_sizes)
  nrow=len(yvals_train)
  num_output=num_classes

  input_X = tf.placeholder('float32',shape =(None,num_features),name="input_X")
  input_y = tf.placeholder('float32',shape = (None,num_classes),name='input_Y')

  s=tf.compat.v1.InteractiveSession()
  ## Weights initialized by random normal function with std_dev = 1/sqrt(number of input features)
  weights_0 = tf.Variable(tf.random.normal([num_features,num_layers_0], stddev=(1/tf.sqrt(float(num_features)))))
  bias_0 = tf.Variable(tf.random.normal([num_layers_0]))
  weights_1 = tf.Variable(tf.random.normal([num_layers_0,num_output], stddev=(1/tf.sqrt(float(num_layers_0)))))
  bias_1 = tf.Variable(tf.random.normal([num_output]))

  ## Initializing weigths and biases
  hidden_output_0 = tf.nn.relu(tf.matmul(input_X,weights_0)+bias_0)
  predicted_y = tf.sigmoid(tf.matmul(hidden_output_0,weights_1) + bias_1)

  ##calculate penalty terms
  series = pd.Series(sensor_sizes)
  cumsum = series.cumsum()
  penalty=(tf.reduce_sum(tf.square(weights_0[0:sensor_sizes[0]])))**0.5
  for i in range(len(sensor_sizes)-1):
    penalty=penalty+(tf.reduce_sum(tf.square(weights_0[cumsum[i]:cumsum[i+1]])))**0.5

  cumsum =[0]+ list(series.cumsum())
  red=0
  r_mat=np.array(xvals_train.corr())
  rsq_mat=[[elem*elem for elem in inner] for inner in r_mat]
  rsq_mat=pd.DataFrame(rsq_mat)
  for i in range(len(sensor_sizes)):
    for j in range(len(sensor_sizes)):
      if j!=i:
        red=red+dep(rsq_mat,sensor_sizes,i+1,j+1)*((tf.reduce_sum(tf.square(weights_0[cumsum[j]:cumsum[j+1]])))*(tf.reduce_sum(tf.square(weights_0[cumsum[i]:cumsum[i+1]])))**0.5)

  if len(sensor_sizes)>1:
    red=red/(len(sensor_sizes)*(len(sensor_sizes)-1))

  loss = tf.reduce_mean(tf.square(predicted_y-tf.convert_to_tensor(yvals_train, dtype=tf.float32))) + regularizer_rate_0*red + regularizer_rate_1*penalty 


  ## Variable learning rate
  learning_rate = tf.train.exponential_decay(starter_learning_rate, 0, 5, 0.85, staircase=True)
  ## Adam optimzer for finding the right weight
  optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss,var_list=[weights_0,weights_1,
                                                                         bias_0,bias_1])    
  ## Metrics definition
  correct_prediction = tf.equal(tf.argmax(yvals_train,1), tf.argmax(predicted_y,1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  training_accuracy = []
  training_loss = []
  testing_accuracy = []
  #s.run(tf.initialize_all_variables)
  s.run(tf.compat.v1.global_variables_initializer())
  for epoch in range(epochs):    
    arr = np.arange(nrow)
    np.random.shuffle(arr)
    for index in range(0,nrow,batch_size):
        s.run(optimizer, {input_X: xvals_train,
                          input_y: yvals_train})
        
    training_accuracy.append(s.run(accuracy, feed_dict= {input_X:xvals_train, 
                                                         input_y: yvals_train}))
    training_loss.append(s.run(loss, {input_X: xvals_train, 
                                      input_y: yvals_train}))
    print("Epoch:{0}, Train loss: {1:.2f} Train acc: {2:.3f}".format(epoch,
                                                                    training_loss[epoch],
                                                                    training_accuracy[epoch]))
   
  ## Evaluation of test data
  for i in range(len(xvals_test)):
    print('Actual:', yvals_test[i], 'Predicted:', np.rint(s.run(predicted_y, feed_dict={input_X: [xvals_test.iloc[i]]})))

  y_pred = np.rint(s.run(predicted_y, feed_dict={input_X: xvals_test}))

  testacc = accuracy_score(yvals_test, y_pred)
 
  print("\nTest Accuracy: {0:f}\n".format(testacc))

  w0=weights_0.eval()
  w=[]
  #w.append(norm(w0[0:sensor_sizes[0]],2))
  for i in range(len(sensor_sizes)):
    w.append(norm(w0[cumsum[i]:cumsum[i+1]],2))
  print(w)
  #Feature selection
  if reduction==True:
    v=[i for i,x in enumerate(w) if x > 0.1*max(w)]
    selected=[]
    for i in v:
      selected.append(xvals.iloc[:,range(cumsum[i],cumsum[i+1])])
    print(selected[0])
    xvals_reduced=pd.concat(selected,ignore_index=True, axis=1)
    print(xvals_reduced)
    acc=0
    sensor_sizes_red=[sensor_sizes[i] for i in v]
    for i in range(10):
      acc=acc+func_modified2(regularizer_rate_0,regularizer_rate_1,num_layers_0, epochs, batch_size, num_classes, sensor_sizes_red,dep_cor,xvals_reduced,yvals,reduction=False)
    return(acc/10)
  else:
    return(testacc)

In [ ]:
func_modified2(0.1,0.11,10,500,10,3,[2,2],dep_cor,xvals,yvals,False)

/usr/local/lib/python3.8/dist-packages/tensorflow/python/client/session.py:1768: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Epoch:0, Train loss: 1.48 Train acc: 0.333
Epoch:1, Train loss: 1.42 Train acc: 0.333
Epoch:2, Train loss: 1.38 Train acc: 0.333
Epoch:3, Train loss: 1.33 Train acc: 0.417
Epoch:4, Train loss: 1.28 Train acc: 0.600
Epoch:5, Train loss: 1.24 Train acc: 0.658
Epoch:6, Train loss: 1.20 Train acc: 0.667
Epoch:7, Train loss: 1.16 Train acc: 0.675
Epoch:8, Train loss: 1.12 Train acc: 0.675
Epoch:9, Train loss: 1.08 Train acc: 0.675
Epoch:10, Train loss: 1.05 Train acc: 0.683
Epoch:11, Train loss: 1.01 Train acc: 0.767
Epoch:12, Train loss: 0.98 Train acc: 0.933
Epoch:13, Train loss: 0.96 Train acc: 0.967
Epoch:14, Train loss: 0.93 Train acc: 0.925
Epoch:15, Train loss: 0.91 Train acc: 0.875
Epoch:16, Train loss: 0.89 Train acc: 0.858
Epoch:17, Train loss: 0.86 Train acc: 0.850
Epoch:18, Train loss: 0.84 Train acc: 0.842
Epoch:19, Train loss: 0.82 Train acc: 0.842
Epoch:20, Train loss: 0.80 Train acc: 0.842
Epoch:21, Train loss: 0.79 Train acc: 0.817
Epoch:22, Train loss: 0.77 Train acc: 0.81

0.9666666666666667

In [ ]:
def func_modified3(regularizer_rate_0,regularizer_rate_1,num_layers_0, epochs, batch_size, num_classes, sensor_sizes, dep, xvals, yvals, reduction):

  from numpy.linalg import norm
  import keras
  from keras.layers import Activation, Dense
  import numpy as np
  import pandas as pd
  import tensorflow as tf
  from sklearn.metrics import accuracy_score
  from tensorflow.keras.layers import Dense, Activation
  import tensorflow.compat.v1 as tf
  tf.disable_v2_behavior()
  from sklearn.model_selection import train_test_split

  xvals_train, xvals_test,yvals_train, yvals_test = train_test_split(xvals,yvals,random_state=None, test_size=0.2,  shuffle=True)
                                                                     
  starter_learning_rate = 0.001
  num_features=sum(sensor_sizes)
  nrow=len(yvals_train)
  num_output=num_classes

  input_X = tf.placeholder('float32',shape =(None,num_features),name="input_X")
  input_y = tf.placeholder('float32',shape = (None,num_classes),name='input_Y')

  s=tf.compat.v1.InteractiveSession()
  ## Weights initialized by random normal function with std_dev = 1/sqrt(number of input features)
  weights_0 = tf.Variable(tf.random.normal([num_features,num_layers_0], stddev=(1/tf.sqrt(float(num_features)))))
  bias_0 = tf.Variable(tf.random.normal([num_layers_0]))
  weights_1 = tf.Variable(tf.random.normal([num_layers_0,num_output], stddev=(1/tf.sqrt(float(num_layers_0)))))
  bias_1 = tf.Variable(tf.random.normal([num_output]))

  ## Initializing weigths and biases
  hidden_output_0 = tf.nn.relu(tf.matmul(input_X,weights_0)+bias_0)
  predicted_y = tf.sigmoid(tf.matmul(hidden_output_0,weights_1) + bias_1)

  ##calculate penalty terms
  series = pd.Series(sensor_sizes)
  cumsum = series.cumsum()
  penalty=(tf.reduce_sum(tf.square(weights_0[0:sensor_sizes[0]])))**0.5/sensor_sizes[0]
  for i in range(len(sensor_sizes)-1):
    penalty=penalty+((tf.reduce_sum(tf.square(weights_0[cumsum[i]:cumsum[i+1]])))**0.5)/sensor_sizes[i+1]

  cumsum =[0]+ list(series.cumsum())
  redund=0
  r_mat=np.array(xvals_train.corr())
  rsq_mat=[[elem*elem for elem in inner] for inner in r_mat]
  rsq_mat=pd.DataFrame(rsq_mat)
  for i in range(len(sensor_sizes)):
    for j in range(len(sensor_sizes)):
      if j!=i:
        redund=redund+dep(rsq_mat,sensor_sizes,i+1,j+1)*((tf.reduce_sum(tf.square(weights_0[cumsum[j]:cumsum[j+1]])))*(tf.reduce_sum(tf.square(weights_0[cumsum[i]:cumsum[i+1]])))**0.5)/(sensor_sizes[i]*sensor_sizes[j])

  if len(sensor_sizes)>1:
    redund=redund/(len(sensor_sizes)*(len(sensor_sizes)-1))

  loss = tf.reduce_mean(tf.square(predicted_y-tf.convert_to_tensor(yvals_train, dtype=tf.float32))) + regularizer_rate_0*redund/num_layers_0**2 + regularizer_rate_1*penalty/num_layers_0 


  ## Variable learning rate
  learning_rate = tf.train.exponential_decay(starter_learning_rate, 0, 5, 0.85, staircase=True)
  ## Adam optimzer for finding the right weight
  optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss,var_list=[weights_0,weights_1,
                                                                         bias_0,bias_1])    
  ## Metrics definition
  correct_prediction = tf.equal(tf.argmax(yvals_train,1), tf.argmax(predicted_y,1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  training_accuracy = []
  training_loss = []
  testing_accuracy = []
  #s.run(tf.initialize_all_variables)
  s.run(tf.compat.v1.global_variables_initializer())
  #x=func_modified(regularizer_rate_0,regularizer_rate_1,num_layers_0, epochs, batch_size, num_classes, sensor_sizes, dep)
  #weights_0=x[0]
  #weights_1=x[1]
  #bias_0=x[2]
  #bias_1=x[3]
  for epoch in range(epochs):    
    arr = np.arange(nrow)
    np.random.shuffle(arr)
    for index in range(0,nrow,batch_size):
        s.run(optimizer, {input_X: xvals_train,
                          input_y: yvals_train})
        
    training_accuracy.append(s.run(accuracy, feed_dict= {input_X:xvals_train, 
                                                         input_y: yvals_train}))
    training_loss.append(s.run(loss, {input_X: xvals_train, 
                                      input_y: yvals_train}))
    #print("Epoch:{0}, Train loss: {1:.2f} Train acc: {2:.3f}".format(epoch,training_loss[epoch],training_accuracy[epoch]))
   
  ## Evaluation of test data
  #for i in range(len(xvals_test)):
    #print('Actual:', yvals_test[i], 'Predicted:', np.rint(s.run(predicted_y, feed_dict={input_X: [xvals_test.iloc[i]]})))

  y_pred = np.rint(s.run(predicted_y, feed_dict={input_X: xvals_test}))

  testacc = accuracy_score(yvals_test, y_pred)
 
  print("\nTest Accuracy: {0:f}\n".format(testacc))

  w0=weights_0.eval()
  w=[]
  #w.append(norm(w0[0:sensor_sizes[0]],2))
  for i in range(len(sensor_sizes)):
    w.append(norm(w0[cumsum[i]:cumsum[i+1]],2))
  #Feature selection
  if reduction==True:
    v=[i for i,x in enumerate(w) if x > 0.1*max(w)]
    selected=[]
    for i in v:
      selected.append(xvals.iloc[:,range(cumsum[i],cumsum[i+1])])
  
    xvals_reduced=pd.concat(selected,ignore_index=True, axis=1)
    #print(xvals_reduced)
    acc=0
    sensor_sizes_red=[sensor_sizes[i] for i in v]
    for i in range(10):
      acc=acc+func_modified3(regularizer_rate_0,regularizer_rate_1,num_layers_0, epochs, batch_size, num_classes, sensor_sizes_red,dep_cor,xvals_reduced,yvals,reduction=False)[0]
    return([acc/10,len(sensor_sizes_red),v])
  else:
    return([testacc,len(sensor_sizes)])

In [ ]:
func_modified3(1,1,10,500,10,3,[2,2],dep_cor,xvals,yvals,False)

/usr/local/lib/python3.8/dist-packages/tensorflow/python/client/session.py:1768: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '



Test Accuracy: 1.000000



[1.0, 2]

In [55]:
def func_modified3(regularizer_rate_0,regularizer_rate_1,num_layers_0, epochs, batch_size, num_classes, sensor_sizes, dep, xvals, yvals, reduction):

  from numpy.linalg import norm
  import keras
  from keras.layers import Activation, Dense
  import numpy as np
  import pandas as pd
  import tensorflow as tf
  from sklearn.metrics import accuracy_score
  from tensorflow.keras.layers import Dense, Activation
  import tensorflow.compat.v1 as tf
  tf.disable_v2_behavior()
  from sklearn.model_selection import train_test_split
  

  xvals_train, xvals_test,yvals_train, yvals_test = train_test_split(xvals,yvals,random_state=None, test_size=0.2,  shuffle=True)
                                                                     
  starter_learning_rate = 0.001
  num_features=sum(sensor_sizes)
  nrow=len(yvals_train)
  num_output=num_classes

  input_X = tf.placeholder('float32',shape =(None,num_features),name="input_X")
  input_y = tf.placeholder('float32',shape = (None,num_classes),name='input_Y')

  s=tf.compat.v1.InteractiveSession()
  ## Weights initialized by random normal function with std_dev = 1/sqrt(number of input features)
  weights_0 = tf.Variable(tf.random.normal([num_features,num_layers_0], stddev=(1/tf.sqrt(float(num_features)))))
  bias_0 = tf.Variable(tf.random.normal([num_layers_0]))
  weights_1 = tf.Variable(tf.random.normal([num_layers_0,num_output], stddev=(1/tf.sqrt(float(num_layers_0)))))
  bias_1 = tf.Variable(tf.random.normal([num_output]))

  ## Initializing weigths and biases
  hidden_output_0 = tf.nn.relu(tf.matmul(input_X,weights_0)+bias_0)
  predicted_y = tf.sigmoid(tf.matmul(hidden_output_0,weights_1) + bias_1)

  ##calculate penalty terms
  series = pd.Series(sensor_sizes)
  cumsum = series.cumsum()
  penalty=(tf.reduce_sum(tf.square(weights_0[0:sensor_sizes[0]])))**0.5/sensor_sizes[0]
  for i in range(len(sensor_sizes)-1):
    penalty=penalty+((tf.reduce_sum(tf.square(weights_0[cumsum[i]:cumsum[i+1]])))**0.5)/sensor_sizes[i+1]

  cumsum =[0]+ list(series.cumsum())
  redund=0
  r_mat=np.array(xvals_train.corr())
  rsq_mat=[[elem*elem for elem in inner] for inner in r_mat]
  rsq_mat=pd.DataFrame(rsq_mat)
  for i in range(len(sensor_sizes)):
    for j in range(len(sensor_sizes)):
      if j!=i:
        redund=redund+dep(rsq_mat,sensor_sizes,i+1,j+1)*((tf.reduce_sum(tf.square(weights_0[cumsum[j]:cumsum[j+1]])))*(tf.reduce_sum(tf.square(weights_0[cumsum[i]:cumsum[i+1]])))**0.5)/(sensor_sizes[i]*sensor_sizes[j])

  if len(sensor_sizes)>1:
    redund=redund/(len(sensor_sizes)*(len(sensor_sizes)-1))

  loss = tf.reduce_mean(tf.square(predicted_y-tf.convert_to_tensor(yvals_train, dtype=tf.float32))) + regularizer_rate_0*redund/num_layers_0**2 + regularizer_rate_1*penalty/num_layers_0 


  ## Variable learning rate
  learning_rate = tf.train.exponential_decay(starter_learning_rate, 0, 5, 0.85, staircase=True)
  ## Adam optimzer for finding the right weight
  optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss,var_list=[weights_0,weights_1,
                                                                         bias_0,bias_1])    
  ## Metrics definition
  correct_prediction = tf.equal(tf.argmax(yvals_train,1), tf.argmax(predicted_y,1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  training_accuracy = []
  training_loss = []
  testing_accuracy = []
  #s.run(tf.initialize_all_variables)
  s.run(tf.compat.v1.global_variables_initializer())
  w0=weights_0.eval()
  print(w0)
  x=func_modified(0,0,num_layers_0, epochs, batch_size, num_classes, sensor_sizes, dep)
  weights_0=x[0]
  weights_1=x[1]
  bias_0=x[2]
  bias_1=x[3]
  #w0=weights_0.eval()
  #print(w0)
  for epoch in range(epochs):    
    arr = np.arange(nrow)
    np.random.shuffle(arr)
    for index in range(0,nrow,batch_size):
        s.run(optimizer, {input_X: xvals_train,
                          input_y: yvals_train})
        
    training_accuracy.append(s.run(accuracy, feed_dict= {input_X:xvals_train, 
                                                         input_y: yvals_train}))
    training_loss.append(s.run(loss, {input_X: xvals_train, 
                                      input_y: yvals_train}))
    #print("Epoch:{0}, Train loss: {1:.2f} Train acc: {2:.3f}".format(epoch,training_loss[epoch],training_accuracy[epoch]))
   
  ## Evaluation of test data
  #for i in range(len(xvals_test)):
    #print('Actual:', yvals_test[i], 'Predicted:', np.rint(s.run(predicted_y, feed_dict={input_X: [xvals_test.iloc[i]]})))

  y_pred = np.rint(s.run(predicted_y, feed_dict={input_X: xvals_test}))

  testacc = accuracy_score(yvals_test, y_pred)
 
  print("\nTest Accuracy: {0:f}\n".format(testacc))

  w0=weights_0.eval()
  w=[]
  
  print(w0)
  #w.append(norm(w0[0:sensor_sizes[0]],2))
  for i in range(len(sensor_sizes)):
    w.append(norm(w0[cumsum[i]:cumsum[i+1]],2))
  #Feature selection
  if reduction==True:
    v=[i for i,x in enumerate(w) if x > 0.1*max(w)]
    selected=[]
    for i in v:
      selected.append(xvals.iloc[:,range(cumsum[i],cumsum[i+1])])
  
    xvals_reduced=pd.concat(selected,ignore_index=True, axis=1)
    #print(xvals_reduced)
    acc=0
    sensor_sizes_red=[sensor_sizes[i] for i in v]
    for i in range(10):
      acc=acc+func_modified3(regularizer_rate_0,regularizer_rate_1,num_layers_0, epochs, batch_size, num_classes, sensor_sizes_red,dep_cor,xvals_reduced,yvals,reduction=False)[0]
    return([acc/10,len(sensor_sizes_red),v])
  else:
    return([testacc,len(sensor_sizes)])

In [58]:
x=func_modified(1,1,10,100,10,3,[2,2],dep_cor)

x[0]

/usr/local/lib/python3.8/dist-packages/tensorflow/python/client/session.py:1768: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Epoch:0, Train loss: 9.29 Train acc: 0.317
Epoch:1, Train loss: 8.93 Train acc: 0.317
Epoch:2, Train loss: 8.58 Train acc: 0.317
Epoch:3, Train loss: 8.25 Train acc: 0.317
Epoch:4, Train loss: 7.95 Train acc: 0.317
Epoch:5, Train loss: 7.67 Train acc: 0.300
Epoch:6, Train loss: 7.40 Train acc: 0.308
Epoch:7, Train loss: 7.16 Train acc: 0.367
Epoch:8, Train loss: 6.92 Train acc: 0.367
Epoch:9, Train loss: 6.70 Train acc: 0.450
Epoch:10, Train loss: 6.49 Train acc: 0.633
Epoch:11, Train loss: 6.28 Train acc: 0.675
Epoch:12, Train loss: 6.09 Train acc: 0.683
Epoch:13, Train loss: 5.91 Train acc: 0.683
Epoch:14, Train loss: 5.73 Train acc: 0.683
Epoch:15, Train loss: 5.56 Train acc: 0.683
Epoch:16, Train loss: 5.40 Train acc: 0.683
Epoch:17, Train loss: 5.25 Train acc: 0.683
Epoch:18, Train loss: 5.10 Train acc: 0.683
Epoch:19, Train loss: 4.96 Train acc: 0.683
Epoch:20, Train loss: 4.83 Train acc: 0.683
Epoch:21, Train loss: 4.70 Train acc: 0.683
Epoch:22, Train loss: 4.57 Train acc: 0.68

<tf.Variable 'Variable_156:0' shape=(4, 10) dtype=float32_ref>

In [ ]:
def func_modified4(regularizer_rate_0,regularizer_rate_1,num_layers_0, epochs, batch_size, num_classes, sensor_sizes, dep, xvals, yvals, reduction):

  from numpy.linalg import norm
  import keras
  from keras.layers import Activation, Dense
  import numpy as np
  import pandas as pd
  import tensorflow as tf
  from sklearn.metrics import accuracy_score
  from tensorflow.keras.layers import Dense, Activation
  import tensorflow.compat.v1 as tf
  tf.disable_v2_behavior()
  from sklearn.model_selection import train_test_split

  xvals_train, xvals_test,yvals_train, yvals_test = train_test_split(xvals,yvals,random_state=None, test_size=0.2,  shuffle=True)
                                                                     
  starter_learning_rate = 0.001
  num_features=sum(sensor_sizes)
  nrow=len(yvals_train)
  num_output=num_classes

  input_X = tf.placeholder('float32',shape =(None,num_features),name="input_X")
  input_y = tf.placeholder('float32',shape = (None,num_classes),name='input_Y')

  s=tf.compat.v1.InteractiveSession()
  ## Weights initialized by random normal function with std_dev = 1/sqrt(number of input features)
  weights_0 = tf.Variable(tf.random.normal([num_features,num_layers_0], stddev=(1/tf.sqrt(float(num_features)))))
  bias_0 = tf.Variable(tf.random.normal([num_layers_0]))
  weights_1 = tf.Variable(tf.random.normal([num_layers_0,num_output], stddev=(1/tf.sqrt(float(num_layers_0)))))
  bias_1 = tf.Variable(tf.random.normal([num_output]))

  ## Initializing weigths and biases
  hidden_output_0 = tf.nn.relu(tf.matmul(input_X,weights_0)+bias_0)
  predicted_y = tf.sigmoid(tf.matmul(hidden_output_0,weights_1) + bias_1)

  ##calculate penalty terms
  series = pd.Series(sensor_sizes)
  cumsum = series.cumsum()
  penalty=(tf.reduce_sum(tf.square(weights_0[0:sensor_sizes[0]])))**0.5/sensor_sizes[0]
  for i in range(len(sensor_sizes)-1):
    penalty=penalty+((tf.reduce_sum(tf.square(weights_0[cumsum[i]:cumsum[i+1]])))**0.5)/sensor_sizes[i+1]

  cumsum =[0]+ list(series.cumsum())
  redund=0
  r_mat=np.array(xvals_train.corr())
  rsq_mat=[[elem*elem for elem in inner] for inner in r_mat]
  rsq_mat=pd.DataFrame(rsq_mat)
  for i in range(len(sensor_sizes)):
    for j in range(len(sensor_sizes)):
      if j!=i:
        redund=redund+dep(rsq_mat,sensor_sizes,i+1,j+1)*((tf.reduce_sum(tf.square(weights_0[cumsum[j]:cumsum[j+1]])))*(tf.reduce_sum(tf.square(weights_0[cumsum[i]:cumsum[i+1]])))**0.5)/(sensor_sizes[i]*sensor_sizes[j])

  if len(sensor_sizes)>1:
    redund=redund/(len(sensor_sizes)*(len(sensor_sizes)-1))

  loss = tf.reduce_mean(tf.square(predicted_y-tf.convert_to_tensor(yvals_train, dtype=tf.float32))) + regularizer_rate_0*redund/num_layers_0**2 + regularizer_rate_1*penalty/num_layers_0 


  ## Variable learning rate
  learning_rate = tf.train.exponential_decay(starter_learning_rate, 0, 5, 0.85, staircase=True)
  ## Adam optimzer for finding the right weight
  optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss,var_list=[weights_0,weights_1,
                                                                         bias_0,bias_1])    
  ## Metrics definition
  correct_prediction = tf.equal(tf.argmax(yvals_train,1), tf.argmax(predicted_y,1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  training_accuracy = []
  training_loss = []
  testing_accuracy = []
  #s.run(tf.initialize_all_variables)
  s.run(tf.compat.v1.global_variables_initializer())
  for epoch in range(epochs):    
    arr = np.arange(nrow)
    np.random.shuffle(arr)
    for index in range(0,nrow,batch_size):
        s.run(optimizer, {input_X: xvals_train,
                          input_y: yvals_train})
        
    training_accuracy.append(s.run(accuracy, feed_dict= {input_X:xvals_train, 
                                                         input_y: yvals_train}))
    training_loss.append(s.run(loss, {input_X: xvals_train, 
                                      input_y: yvals_train}))
    print("Epoch:{0}, Train loss: {1:.2f} Train acc: {2:.3f}".format(epoch,
                                                                    training_loss[epoch],
                                                                    training_accuracy[epoch]))
   
  ## Evaluation of test data
  for i in range(len(xvals_test)):
    print('Actual:', yvals_test[i], 'Predicted:', np.rint(s.run(predicted_y, feed_dict={input_X: [xvals_test.iloc[i]]})))

  y_pred = np.rint(s.run(predicted_y, feed_dict={input_X: xvals_test}))

  testacc = accuracy_score(yvals_test, y_pred)
 
  print("\nTest Accuracy: {0:f}\n".format(testacc))

  w0=weights_0.eval()
  w=[]
  #w.append(norm(w0[0:sensor_sizes[0]],2))
  for i in range(len(sensor_sizes)):
    w.append(norm(w0[cumsum[i]:cumsum[i+1]],2))
  print(w)
  #Feature selection
  if reduction==True:
    v=[i for i,x in enumerate(w) if x > 0.1*max(w)]
    selected=[]
    for i in v:
      selected.append(xvals.iloc[:,range(cumsum[i],cumsum[i+1])])
    print(selected[0])
    xvals_reduced=pd.concat(selected,ignore_index=True, axis=1)
    print(xvals_reduced)
    acc=0
    sensor_sizes_red=[sensor_sizes[i] for i in v]
    for i in range(10):
      acc=acc+func_modified4(regularizer_rate_0,regularizer_rate_1,num_layers_0, epochs, batch_size, num_classes, sensor_sizes_red,dep_cor,xvals_reduced,yvals,reduction=False)
    return(acc/10)
  else:
    return(testacc)

In [ ]:
from scipy.stats import pearsonr
import numpy as np

#function for dependency between m th and n th sensor

def dep_cor(rsq_mat,sensor,m,n):
  
  series = pd.Series(sensor)
  cumsum = list(series.cumsum())
  cumsum=[0]+cumsum
  ind1=list(range(cumsum[m-1],cumsum[m]))
  ind2=list(range(cumsum[n-1],cumsum[n]))
  cor=rsq_mat.iloc[ind1,ind2]
  return(min(cor.apply(max,1)))


In [ ]:
func_modified3(2,1,10,500,10,3,[2,2],dep_cor,xvals,yvals,False)

/usr/local/lib/python3.8/dist-packages/tensorflow/python/client/session.py:1768: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


[[-0.19708167  0.1978043   0.06414646 -0.95133424 -0.11305457 -0.42700055
   0.29217947 -0.8193025   0.08104351 -0.27448344]
 [-0.4879038  -0.18215826 -0.14601058 -0.4990794   0.61015725  0.02099038
   0.00872037 -0.17104909  0.42494467 -0.26559058]
 [-0.2840223  -0.11200962  0.6110784   0.7704659  -0.0853888   0.4914627
   0.39801386 -0.595136   -0.4015481   0.09508768]
 [-0.8637055  -0.02585068  0.27894333 -0.5665355   0.29940957 -0.07858607
   0.3633453  -0.580989    0.6184      0.30730468]]
Epoch:0, Train loss: 0.36 Train acc: 0.317
Epoch:1, Train loss: 0.33 Train acc: 0.317
Epoch:2, Train loss: 0.31 Train acc: 0.317
Epoch:3, Train loss: 0.30 Train acc: 0.317
Epoch:4, Train loss: 0.29 Train acc: 0.317
Epoch:5, Train loss: 0.28 Train acc: 0.308
Epoch:6, Train loss: 0.27 Train acc: 0.300
Epoch:7, Train loss: 0.26 Train acc: 0.275
Epoch:8, Train loss: 0.24 Train acc: 0.283
Epoch:9, Train loss: 0.22 Train acc: 0.400
Epoch:10, Train loss: 0.21 Train acc: 0.367
Epoch:11, Train loss: 0.20

FailedPreconditionError: ignored

In [ ]:
f1=iris['SepalLengthCm']
f2=iris['SepalWidthCm']
f3=iris['PetalLengthCm']
f4=iris['PetalWidthCm']
e1=f1+np.random.normal(loc=0.0, scale=0.05, size=150)
e2=f3+np.random.normal(loc=0.0, scale=0.05, size=150)
e3=f4+np.random.normal(loc=0.0, scale=0.05, size=150)
xvals=pd.concat([f1,f2,e1,f3,f4,e2,e3], axis=1, ignore_index=True).astype(np.float32)


In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for the number of nodes in the hidden layer
param_grid = {'hidden_layer_sizes': np.arange(1, 20, 2)}

# Create a MLPClassifier object
mlp = MLPClassifier(max_iter=1000)

# Create a GridSearchCV object
grid_search = GridSearchCV(mlp, param_grid, cv=10, scoring='accuracy',verbose = 3)

# Fit the GridSearchCV object to the data
grid_search.fit(xvals, yvals)

# Print the best number of nodes in the hidden layer
print("Best number of nodes in hidden layer: ", grid_search.best_params_)
from itertools import repeat
result=[]
for i in [0,2,5]:
  for j in [0,2,5]:
    x=func_modified3(i,j,grid_search.best_params_['hidden_layer_sizes'],400,10,3,[2,3,2],dep_cor,xvals,yvals,True)
    result.append([i,j,x[0],x[1],x[2]])

result=pd.DataFrame(result)
result.columns =["Lambda","Mu", "Test Accuracy", "Number of sensors selected","Selected sensors"]
result

Fitting 10 folds for each of 10 candidates, totalling 100 fits


/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 1/10] END .............hidden_layer_sizes=1;, score=1.000 total time=   0.5s


/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 2/10] END .............hidden_layer_sizes=1;, score=1.000 total time=   0.5s


...

/usr/local/lib/python3.8/dist-packages/tensorflow/python/client/session.py:1768: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '



Test Accuracy: 0.933333



/usr/local/lib/python3.8/dist-packages/tensorflow/python/client/session.py:1768: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '



Test Accuracy: 0.966667



,Lambda,Mu,Test Accuracy,Number of sensors selected,Selected sensors
0,0,0,0.953333,3,"[0, 1, 2]"
1,0,2,0.940000,1,[1]
2,0,5,0.930000,1,[1]
3,2,0,0.953333,2,"[0, 1]"
4,2,2,0.956667,1,[1]
5,2,5,0.966667,1,[1]
6,5,0,0.973333,1,[1]
7,5,2,0.970000,1,[1]
8,5,5,0.970000,1,[1]


In [ ]:
result_iris2=result

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/datasets/GasSensor(cleaned in R).csv',header=None)
data.drop(0,axis=1,inplace=True)
data.drop(0,axis=0,inplace=True)
data.dropna() 
data[1]=data[1].replace(['1'],1)
data[1]=data[1].replace(['2'],2)
data[1]=data[1].replace(['3'],3)
data[1]=data[1].replace(['4'],4)
data[1]=data[1].replace(['5'],5)
data[1]=data[1].replace(['6'],6)

yvals=data[1]
yvals=to_categorical(np.asarray(yvals.factorize()[0]))

xvals=data.iloc[:,1:129].astype(np.float32)
from scipy.stats import zscore
for i in range(128):
  xvals[i+2]=zscore(xvals[i+2])
xvals


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,2,3,4,5,6,7,8,9,10,11,...,120,121,122,123,124,125,126,127,128,129
1,-0.498823,-0.353696,-0.599967,-0.640209,-0.508537,0.504313,0.519048,0.420417,-0.656019,-0.313086,...,0.925279,0.607139,-1.165980,-0.452359,-1.259691,-1.141841,-1.226102,1.038312,0.951309,0.641065
2,-0.344104,-0.304448,-0.427355,-0.491356,-0.504699,0.348537,0.402966,0.400031,-0.522842,-0.287665,...,0.875453,0.646466,-1.086196,-0.385757,-1.173395,-1.069507,-1.190985,0.956254,0.896151,0.684262
3,-0.119290,-0.236096,-0.269089,-0.330763,-0.400975,0.117085,0.231118,0.364061,-0.308821,-0.244310,...,0.782796,0.613568,-0.964482,-0.145462,-1.092653,-1.001250,-1.115337,0.850126,0.808405,0.655673
4,-0.108947,-0.236318,-0.046722,-0.099488,0.341659,0.102857,0.221204,0.366421,-0.296068,-0.243619,...,0.758672,0.606373,-0.948180,-0.123006,-0.982531,-0.913377,-1.045200,0.826851,0.788437,0.648965
5,0.110479,-0.181176,-0.084136,-0.121640,-0.242369,-0.151210,0.038570,0.332179,-0.083764,-0.203216,...,0.651558,0.563989,-0.807353,0.237454,-0.980268,-0.892513,-1.020654,0.708167,0.685640,0.572631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13906,-0.307916,-0.246847,-0.485160,-0.549709,-0.458386,0.414751,0.421862,0.335800,-0.236345,-0.209829,...,0.879392,0.552717,-0.994872,-0.693892,-1.160764,-1.036392,-0.955375,0.967338,0.885127,0.555494
13907,-0.143795,-0.249012,-0.151498,-0.207163,-0.155137,0.077420,0.166321,0.214673,-0.265725,-0.260120,...,0.130994,0.120197,-0.279282,-0.184373,-0.375570,-0.477159,-0.521098,0.096686,0.067103,0.158543
13908,1.901950,0.664126,1.848079,1.812697,1.423620,-1.601135,-1.559541,-2.055822,2.693727,0.505053,...,-2.643781,-1.644654,2.188036,2.528193,1.933941,0.953627,0.991928,-2.603609,-2.664302,-1.920199
13909,-0.306050,-0.301808,-0.353361,-0.399921,-0.338603,0.244922,0.297209,0.287035,-0.451244,-0.293780,...,0.471342,0.329141,-0.672849,-0.545487,-0.737872,-0.697882,-0.699366,0.489014,0.449561,0.310945


In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from itertools import repeat
# Define the parameter grid for the number of nodes in the hidden layer
param_grid = {'hidden_layer_sizes': np.arange(1, 20, 2)}

# Create a MLPClassifier object
mlp = MLPClassifier(max_iter=1000)

# Create a GridSearchCV object
grid_search = GridSearchCV(mlp, param_grid, cv=10, scoring='accuracy',verbose = 3)

# Fit the GridSearchCV object to the data
grid_search.fit(xvals, yvals)

# Print the best number of nodes in the hidden layer
print("Best number of nodes in hidden layer: ", grid_search.best_params_)



Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV 1/10] END .............hidden_layer_sizes=1;, score=0.219 total time=  28.1s


/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:699: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


[CV 2/10] END .............hidden_layer_sizes=1;, score=0.598 total time=   9.8s


/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:699: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


[CV 3/10] END .............hidden_layer_sizes=1;, score=0.254 total time=   3.0s


/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:699: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


[CV 4/10] END .............hidden_layer_sizes=1;, score=0.000 total time=   1.0s


/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:699: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


[CV 5/10] END .............hidden_layer_sizes=1;, score=0.345 total time=   1.6s


In [ ]:
from itertools import repeat
result=[]
for i in [0,2,5]:
  for j in [0,2,5]:
    x=func_modified3(i,j,19,400,100,6,list(repeat(8,16)),dep_cor,xvals,yvals,True)
    result.append([i,j,x[0],x[1],x[2]])

result_gs=pd.DataFrame(result)
result_gs.columns =["Lambda","Mu", "Test Accuracy", "Number of sensors selected","Selected sensors"]
result_gs

NameError: ignored

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for the number of nodes in the hidden layer
param_grid = {'hidden_layer_sizes': np.arange(1, 20, 2)}

# Create a MLPClassifier object
mlp = MLPClassifier(max_iter=1000)

# Create a GridSearchCV object
grid_search = GridSearchCV(mlp, param_grid, cv=10, scoring='accuracy',verbose = 3)

# Fit the GridSearchCV object to the data
grid_search.fit(xvals, yvals)

# Print the best number of nodes in the hidden layer
print("Best number of nodes in hidden layer: ", grid_search.best_params_)
from itertools import repeat
for i in 

func_modified2(0.5,0.1,grid_search.best_params_['hidden_layer_sizes'],400,10,3,[2,2],dep_cor,xvals,yvals,True)


Fitting 10 folds for each of 10 candidates, totalling 100 fits


/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:699: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


[CV 1/10] END .............hidden_layer_sizes=1;, score=0.303 total time=   7.5s


/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:699: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


[CV 2/10] END .............hidden_layer_sizes=1;, score=0.461 total time=   1.9s


/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:699: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


[CV 3/10] END .............hidden_layer_sizes=1;, score=0.158 total time=   0.8s


/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:699: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


[CV 4/10] END .............hidden_layer_sizes=1;, score=0.000 total time=   1.6s


In [ ]:
np.arange(2, 20, 2)

array([ 2,  4,  6,  8, 10, 12, 14, 16, 18])